**Basic DENSENET**

---



In [5]:
from tensorflow.keras.datasets import cifar10
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Conv2D, AveragePooling2D, Activation, concatenate, GlobalAveragePooling2D, Dropout
from tensorflow.keras import backend, Model

In [6]:
def get_cifar(dataset):
  (x_train, y_train), (x_test, y_test) = cifar10.load_data()
  x_train = x_train/255.0
  x_test = x_test/255.0
  input_shape = (32, 32, 3)
  return x_train, y_train, x_test, y_test, input_shape

x_train, y_train, x_test, y_test, input_shape = get_cifar(cifar10)
print(x_train.shape, y_train.shape)

(50000, 32, 32, 3) (50000, 1)


This time we will build using the Functional API of keras. This gives us 
more flexibility as compared to the sequential API.

---

1. As per the original paper, we implemented a **composite function** containing BatchNormalization, Relu activation and a conv2D layer with kernel size 3.

2. Our **transition layers **are implemented with BatchNormalization, 1 X 1 conv2D layer and Average pooling. Transition layers also use a compression factor of 0.5.

3. We use same padding for every convolutional layer.

4. Before entering the first dense block we begin with a conv2D layer with 
16 filters of kernel size 3.

In [7]:
class DenseNet:
    def __init__(self, inp_shape=None, num_blocks=None, nb_classes=None, growth_rate=16):
        """
        :param inp_shape: input shape of cifar10 which is (32, 32, 3)
        :param num_blocks: number of dense blocks to use. should be at least 5
        :nb_classes: number of classes in dataset, eg 10 for cifar10
        :growth_rate = number of filters per dense block
        """
        self.inp_shape = inp_shape
        self.num_block = num_blocks
        self.nb_classes = nb_classes
        self.growth_rate = growth_rate

        # checks
        if self.nb_classes is None:
            raise Exception('Please define the number of classes as it is important for the final layer')

        if self.num_block < 5:
            raise Exception('Number of dense blocks must be an interger greater than or equal 5')

    def dense_block(self, x, nb_chan, block_size):
        for _ in range(block_size):
            convblock = self.composite_fxn(x, nb_chan)
            x = concatenate([convblock, x])
        return x

    def composite_fxn(self, x, num_filter):
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = Conv2D(num_filter, kernel_size=5, padding='same')(x)
       # x = Dropout(0.3)(x)
        return x

    def transition_layer(self, x, compression=0.5):
        x = BatchNormalization()(x)
        x = Conv2D(int(compression * backend.int_shape(x)[-1]), kernel_size=1, padding='same')(x)
        x = AveragePooling2D(2, padding='same')(x)
        return x

    def build_model(self):
        """
        Build a keras DenseNet
        :return: Keras model
        """

        print('Building DenseNet')

        # Initial step before entering dense blocks
        filters = self.growth_rate
        inputs = Input(shape=self.inp_shape)
        x = Conv2D(filters, kernel_size=3, padding='same')(inputs)

        for block in range(0, self.num_block, 2)[1:]:
            x = self.dense_block(x, filters, block)  # ensure every block is a multiple of two
            x = self.transition_layer(x)
            filters += self.growth_rate

        x = GlobalAveragePooling2D()(x)
        output = Dense(self.nb_classes, activation='softmax')(x)

        model = Model(inputs, output)
        model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

        return model

No we train our model. We recommend using GPUs as the network has parameters in the million range

In [9]:
model = DenseNet(inp_shape=input_shape, num_blocks=7, nb_classes=10, growth_rate=16).build_model()
print(model.summary())

model.fit(x_train, y_train, batch_size=64, epochs=20, validation_data=(x_test, y_test))


Building DenseNet
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_17 (Conv2D)              (None, 32, 32, 16)   448         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_16 (BatchNo (None, 32, 32, 16)   64          conv2d_17[0][0]                  
__________________________________________________________________________________________________
activation_12 (Activation)      (None, 32, 32, 16)   0           batch_normalization_16[0][0]     
__________________________________________________________________________